# Libraries

In [ ]:
from zumi.zumi import Zumi
from zumi.protocol import Note 
from zumi.util.camera import Camera
from zumi.util.vision import Vision
from zumi.util.screen import Screen
from zumi.personality import Personality
import time
screen = Screen()
camera = Camera()
vision = Vision()
zumi = Zumi()
personality = Personality(zumi, screen)

# calibrate

In [ ]:
zumi.calibrate_gyro()
print("Done")

In [ ]:
zumi.mpu.calibrate_MPU(count=500)

### Functions Cell

In [ ]:
def forward_move_inches(distance):
    intercept = -0.5913183949807044
    slope = 6.730326311017472
    seconds = (distance - intercept)/slope
    zumi.forward(speed=40,duration=seconds) 

def reverse_move_inches(distance):
    intercept = -0.5913183949807044
    slope = 6.730326311017472
    seconds = (distance - intercept)/slope
    zumi.reverse(speed=40,duration=seconds)  

def move_to_point(point,flag): 
    #flag is either 1 or -1. 1 is up, -1 is down. flag1: office and factory. flag2: school and museum
    x_point = point[0]
    y_point = point[1]
    
    if flag == 1:
        forward_move_inches(x_point)
        zumi.turn(90)
        reverse_move_inches(y_point)
        time.sleep(1)
        forward_move_inches(y_point)
        zumi.turn(0)
        forward_move_inches(35-x_point)
        zumi.turn(-90)
    elif flag == -1:
        forward_move_inches(x_point)
        zumi.turn(-90)
        reverse_move_inches(y_point)
        time.sleep(1)
        forward_move_inches(y_point)
        zumi.turn(0)
        forward_move_inches(35-x_point)
        zumi.turn(-90)

        
        
def doremi():
    zumi.play_note(Note.C4,1000)
    zumi.play_note(Note.D4,1000)
    zumi.play_note(Note.E4,1000)
    
def ff():
    # song: Final Fantasy - Victory Fanfare
    zumi.play_note(Note.C5,166.6666666)
    zumi.play_note(Note.C5,166.6666666)
    zumi.play_note(Note.C5,166.6666666)
    zumi.play_note(Note.C5,500)
    zumi.play_note(Note.GS4,500)
    zumi.play_note(Note.AS4,500)
    zumi.play_note(Note.C5,333.3333333)
    zumi.play_note(Note.AS4,166.6666666)
    zumi.play_note(Note.C5,1000)

def find_thresholds():
    right_th = []
    left_th = []
    for x in range(50):
        ir_readings = zumi.get_all_IR_data()
        bottom_right_ir = ir_readings[1]
        bottom_left_ir = ir_readings[3]

        right_th.append(bottom_right_ir)
        left_th.append(bottom_left_ir)

        time.sleep(0.1)
        
    actual_right_th = round((max(right_th) - min(right_th))/2 + min(right_th))
    actual_left_th = round((max(left_th) - min(left_th))/2 + min(left_th))
    
    #return print("right thresh: ", actual_right_th, "\t left thresh: ", actual_left_th)
    return actual_right_th, actual_left_th    
    


### Global Variables

In [ ]:
messages = []

# Run cell below if u need to update IR threshold values and speed calibration

In [ ]:
find_thresholds()

# R_TH = vals[0]
# L_TH = vals[1]

In [ ]:
zumi.reset_gyro()
zumi.calibrate_gyro()
zumi.speed_calibration(ir_threshold = 168) 

## Run cell below to update ir values

In [ ]:
# newest ir values
R_TH = 112
L_TH = 168


# Challenge Functions

#### Section A Code

In [ ]:
def Chal_A():
    zumi.reset_gyro()
       
    # DON'T TOUCH THE Y-VALUE
#     factory = (6.98,7.09)  
#     school = (14.08,7.09)
#     office = (20,7.09) # original 21.12
#     museum = (25,7.09) # original 28.17

    factory = (7,7)
    school = (14,7)
    office = (21,7)
    museum = (28,7)


    camera.start_camera()

    for i in range(50):
        frame = camera.capture()

        qr_code = vision.find_QR_code(frame)
        message1 = vision.get_QR_message(qr_code)
        
        camera.show_image(frame)
        camera.clear_output()
        print("searching for QR  code...")
        if message1 == "factory" or message1 == "school" or message1 == "office" or message1 == "museum":
            print("found qr code")
            doremi()
            messages.append(message1)
            break
            #camera.close()

    if messages[0] == "factory":
        print("found factory")
        move_to_point(factory,1)
    elif messages[0] == "school":
        print("found school")
        move_to_point(school,-1)
    elif messages[0] == "office":
        print("found office")
        move_to_point(office,1)
    elif messages[0] == "museum":
        print("found museum")
        move_to_point(museum,-1)

    camera.close()
    zumi.stop()
    
    
    new_angle = zumi.read_z_angle()
    print(new_angle)
    forward_move_inches(11);
    zumi.funnel_align(speed=1, duration=7, angle=None, angle_adj=1.1, l_th=L_TH, r_th=R_TH)
    zumi.turn(new_angle)

### Section B

In [ ]:
#Challenge B
#This challenge starts facing the STOP sign, 
#which should be placed at the marker before the start of the run. 
#You can use the "funnel" shape to help get aligned with the STOP sign. 
#Zumi must play the 3-second 'do-re-mi' sound when the STOP sign is detected, then wait. 
#Zumi should only continue when the STOP sign is removed. 
#Then she should drive through segments 1 - 3, without touching the walls or driving onto grass.
     
def Chal_B():
    camera.start_camera()
    for i in range(10):
        image = camera.capture()
        gray = vision.convert_to_gray(image)
        data = vision.find_stop_sign(gray)
        if data is not None: #if stop sign is found
            print("found stop sign")
            zumi.stop()
            doremi()
        else:
            break
        camera.show_image(gray)
        camera.clear_output()
       
    camera.close()
    
    zumi.forward_avoid_collision(40,5,None,180,180)
    zumi.turn(-180)
    zumi.forward_avoid_collision(40,5,None,180,180)
    zumi.turn(-270)
    zumi.forward_avoid_collision(40,5,None,180,180)
    zumi.turn(-180)
    zumi.forward_avoid_collision(40,5,None,180,180)
    zumi.turn(-90)
    
    forward_move_inches(13)
    zumi.funnel_align(speed=1, duration=7, angle=None, angle_adj=1.1, l_th=L_TH, r_th=R_TH)
    zumi.turn(-90)

### Section C 

In [ ]:
#Challenge C
def Chal_C():    
    # zumi should have stopped at end of funnel staring at red/green square
    camera.start_camera()
    for i in range(50):
        image = camera.capture()
        red = vision.find_red_object(image)
        green = vision.find_green_object(image)
        camera.show_image(image)
        if red is not None:
            print('found red square')
            doremi()
        elif green is not None:
            print('found green square')
            zumi.forward()
            zumi.line_follow_gyro_assist(speed=8, duration=45, angle=None, angle_adj=1, l_th=L_TH, r_th=R_TH)
            zumi.turn_left()
            zumi.line_follow_gyro_assist(speed=8, duration=45, angle=None, angle_adj=1, l_th=L_TH, r_th=R_TH)
            zumi.turn_right()
            zumi.line_follow_gyro_assist(speed=8, duration=45, angle=None, angle_adj=1, l_th=L_TH, r_th=R_TH)
            zumi.turn_right()
            break
        else:
            zumi.stop()
        
    camera.close()
    zumi.stop()
           
    # === START OF PART 2 ===
    # ZUMI SHOULD BE AT THE LONG END OF THE T-INTERSECTION
    camera.start_camera()
    for i in range(10):
        frame = camera.capture()
        qr_code = vision.find_QR_code(frame)
        message2 = vision.get_QR_message(qr_code)
        if message2 is not None:
            messages.append(message2)
        camera.show_image(frame)
        camera.clear_output()

        if message2 == "left" or message2 == "right":
            print("I see QR code")
            doremi()
            camera.close()
            break

    if messages[-1] == "left":
        print("I go left")
        zumi.line_follow_gyro_assist(speed=10, duration=30, angle=None, angle_adj=1, l_th=L_TH, r_th=R_TH)
        zumi.turn_left()
        zumi.line_follow_gyro_assist(speed=10, duration=30, angle=None, angle_adj=1, l_th=L_TH, r_th=R_TH)
        zumi.turn_right()
        zumi.line_follow_gyro_assist(speed=10, duration=30, angle=None, angle_adj=1, l_th=L_TH, r_th=R_TH)
        zumi.forward(speed=5)

    elif messages[-1] == "right":
        print("I go right")
        zumi.line_follow_gyro_assist(speed=10, duration=30, angle=None, angle_adj=1, l_th=L_TH, r_th=R_TH)
        zumi.turn_right()
        zumi.line_follow_gyro_assist(speed=10, duration=5, angle=None, angle_adj=1, l_th=L_TH, r_th=R_TH)
        zumi.turn_left()
        zumi.line_follow_gyro_assist(speed=10, duration=30, angle=None, angle_adj=1, l_th=L_TH, r_th=R_TH)
        zumi.forward(speed=5)
        
    personality.happy()
    screen.draw_text_center(messages[0]+" "+messages[1])
    ff()

    print("Done!")
    camera.close()  
    zumi.stop()
    camera.close()
    
    

# Main Body

In [ ]:
start = time.time()
messages = []
Chal_A()
Chal_B()
Chal_C()
end = time.time()
total = end-start
print(total)